In [1]:
import tkinter as tk
from tkinter import messagebox
from ttkbootstrap import Style, ttk
import sqlite3  # Don't forget to import sqlite3

class JonasManagementSystem:
    def __init__(self):
        self.conn = sqlite3.connect('members.db', check_same_thread=False)
        self.cursor = self.conn.cursor()
        self.create_tables()

    def create_tables(self):
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS members (
                member_id TEXT PRIMARY KEY,
                balance REAL
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS transactions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                member_id TEXT,
                amount REAL,
                payment_method TEXT,
                balance_after REAL,
                transaction_time DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        ''')

        self.conn.commit()

    def add_member(self, member_id: str, initial_balance: float):
        try:
            self.cursor.execute("INSERT INTO members (member_id, balance) VALUES (?, ?)", (member_id, initial_balance))
            self.conn.commit()
            return f"Member {member_id} added successfully."
        except sqlite3.IntegrityError:
            return "Member ID already exists."

    def process_payment(self, member_id: str, amount: float, payment_method: str):
        try:
            self.cursor.execute("UPDATE members SET balance = balance + ? WHERE member_id = ?", (amount, member_id))
            self.cursor.execute("SELECT balance FROM members WHERE member_id = ?", (member_id,))
            balance_after = self.cursor.fetchone()[0]
            self.cursor.execute("INSERT INTO transactions (member_id, amount, payment_method, balance_after) VALUES (?, ?, ?, ?)", (member_id, amount, payment_method, balance_after))
            self.conn.commit()
            return f"Payment of {amount} processed successfully."
        except sqlite3.OperationalError:
            return "Database error occurred."

    def get_member_balance(self, member_id: str) -> float:
        self.cursor.execute("SELECT balance FROM members WHERE member_id = ?", (member_id,))
        result = self.cursor.fetchone()
        return result[0] if result else 0.0

    def get_transaction_history(self, member_id: str):
        self.cursor.execute("SELECT * FROM transactions WHERE member_id = ?", (member_id,))
        return self.cursor.fetchall()

    def get_all_transactions(self):
        self.cursor.execute("SELECT * FROM transactions")
        return self.cursor.fetchall()

    def close_connection(self):
        self.conn.close()

class PaymentApp:
    def __init__(self, root):
        self.root = root
        self.style = Style(theme='flatly')  # Use the 'litera' theme for a blue color scheme

        self.root.title("Karen Country Club Payment Integration System")

        self.payment_system = JonasManagementSystem()

        # Create widgets
        self.create_widgets()

    def create_widgets(self):
        # Member ID
        ttk.Label(self.root, text="Member ID:").grid(row=0, column=0, padx=10, pady=10)
        self.member_id_entry = ttk.Entry(self.root)
        self.member_id_entry.grid(row=0, column=1, padx=10, pady=10)

        # Amount
        ttk.Label(self.root, text="Amount:").grid(row=1, column=0, padx=10, pady=10)
        self.amount_entry = ttk.Entry(self.root)
        self.amount_entry.grid(row=1, column=1, padx=10, pady=10)

        # Payment Method
        ttk.Label(self.root, text="Payment Method:").grid(row=2, column=0, padx=10, pady=10)
        self.payment_method = tk.StringVar()
        self.payment_method.set("Credit Card")  # Default value
        payment_methods = ["Credit Card", "Mobile Money", "Cheque", "Cash"]
        self.payment_method_menu = ttk.Combobox(self.root, textvariable=self.payment_method, values=payment_methods)
        self.payment_method_menu.grid(row=2, column=1, padx=10, pady=10)

        # Add Member Button
        self.add_member_button = ttk.Button(self.root, text="Add Member", command=self.add_member, bootstyle="success")
        self.add_member_button.grid(row=5, column=0, columnspan=2, pady=10)

        # Process Payment Button
        self.submit_button = ttk.Button(self.root, text="Process Payment", command=self.process_payment, bootstyle="primary")
        self.submit_button.grid(row=3, column=0, columnspan=2, padx=10, pady=10)
        
        # View All Transactions Button
        self.view_all_button = ttk.Button(self.root, text="View All Transactions", command=self.view_all_transactions, bootstyle="info")
        self.view_all_button.grid(row=7, column=0, columnspan=2, pady=10)

        # View Member Transactions Button
        self.view_member_button = ttk.Button(self.root, text="View Member Transactions", command=self.view_member_transactions, bootstyle="info")
        self.view_member_button.grid(row=6, column=0, columnspan=2, pady=10)

        # Status
        self.status_label = ttk.Label(self.root, text="", foreground="green")
        self.status_label.grid(row=4, column=0, columnspan=2, padx=10, pady=10)

    def process_payment(self):
        try:
            member_id = self.member_id_entry.get()
            amount = float(self.amount_entry.get())
            payment_method = self.payment_method.get()

            result = self.payment_system.process_payment(member_id, amount, payment_method)
            self.status_label.config(text=result)
        except ValueError:
            self.status_label.config(text="Invalid input")

    def add_member(self):
        member_id = self.member_id_entry.get()
        try:
            initial_balance = float(self.amount_entry.get())
        except ValueError:
            self.status_label.config(text="Invalid balance")
            return

        result = self.payment_system.add_member(member_id, initial_balance)
        self.status_label.config(text=result)

    def view_all_transactions(self):
        transactions = self.payment_system.get_all_transactions()

        if not transactions:
            self.status_label.config(text="No transactions found.")
            return

        # Open a new window to show the transaction history
        history_window = tk.Toplevel(self.root)
        history_window.title("All Transactions")

        ttk.Label(history_window, text="Transaction ID").grid(row=0, column=0)
        ttk.Label(history_window, text="Member ID").grid(row=0, column=1)
        ttk.Label(history_window, text="Amount").grid(row=0, column=2)
        ttk.Label(history_window, text="Payment Method").grid(row=0, column=3)
        ttk.Label(history_window, text="Timestamp").grid(row=0, column=4)
        ttk.Label(history_window, text="Balance").grid(row=0, column=5)

        for i, transaction in enumerate(transactions, start=1):
            ttk.Label(history_window, text=transaction[0]).grid(row=i, column=0)  # Transaction ID
            ttk.Label(history_window, text=transaction[1]).grid(row=i, column=1)  # Member ID
            ttk.Label(history_window, text=transaction[2]).grid(row=i, column=2)  # Amount
            ttk.Label(history_window, text=transaction[3]).grid(row=i, column=3)  # Payment Method
            ttk.Label(history_window, text=transaction[4]).grid(row=i, column=4)  
            ttk.Label(history_window, text=transaction[5]).grid(row=i, column=5)  

    def view_member_transactions(self):
        member_id = self.member_id_entry.get()
        transactions = self.payment_system.get_transaction_history(member_id)

        if not transactions:
            self.status_label.config(text="No transactions found for this member.")
            return

        # Open a new window to show the transaction history for the member
        history_window = tk.Toplevel(self.root)
        history_window.title(f"Transactions for {member_id}")

        ttk.Label(history_window, text="Transaction ID").grid(row=0, column=0)
        ttk.Label(history_window, text="Amount").grid(row=0, column=1)
        ttk.Label(history_window, text="Payment Method").grid(row=0, column=2)
        ttk.Label(history_window, text="Timestamp").grid(row=0, column=3)
        ttk.Label(history_window, text="Balance").grid(row=0, column=4)

        for i, transaction in enumerate(transactions, start=1):
            ttk.Label(history_window, text=transaction[0]).grid(row=i, column=0)  # Transaction ID
            ttk.Label(history_window, text=transaction[2]).grid(row=i, column=1)  # Amount
            ttk.Label(history_window, text=transaction[3]).grid(row=i, column=2)  # Payment Method
            ttk.Label(history_window, text=transaction[4]).grid(row=i, column=3)  
            ttk.Label(history_window, text=transaction[5]).grid(row=i, column=4)  

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = PaymentApp(root)
    root.mainloop()
